# DATA620: Project 3 - Gender Identification

## Homework Team 3: David Simbandumwe, Eric Lehmphul and Lidiia Tronina

Your project should be submitted (as an Jupyter Notebook via GitHub). The team members should all submit a link to the same repository. If any of you don't submit the link to GitHub, I will assume you didn't collaborate.
Using any of the three classifiers described in chapter 6 of Natural Language Processing with Python, and any features you can think of, build the best name gender classifier you can. Begin by splitting the Names Corpus into three subsets: 500 words for the test set, 500 words for the dev-test set, and the remaining 6900 words for the training set. Then, starting with the example name gender classifier, make incremental improvements. Use the dev-test set to check your progress. Once you are satisfied with your classifier, check its final performance on the test set. How does the performance on the test set compare to the performance on the dev-test set? Is this what you'd expect?

### Load Required Packages

In [20]:
import nltk
from nltk.corpus import names
from nltk.classify import apply_features

import random

### Functions

In [21]:
# extract features
def gender_features1(name):
    features = {}
    #features["firstletter"] = name[0].lower()
    features["suffix1"] = name[-1].lower()
    features["suffix2"] = name[-2].lower()

    return features

### Import and Lable Names

In [22]:
names = ([(name, 'male') for name in names.words('male.txt')] +
        [(name, 'female') for name in names.words('female.txt')])
random.shuffle(names)

### Split Corpus

In [23]:
train_names = names[1500:]
devtest_names = names[500:1500]
test_names = names[:500]

In [24]:
train_set = apply_features(gender_features1, train_names)
devtest_set = apply_features(gender_features1, devtest_names)
test_set = apply_features(gender_features1, test_names)

### Train Classifier

In [25]:
classifier = nltk.NaiveBayesClassifier.train(train_set) 

### Analyze Errors 

In [26]:
nltk.classify.accuracy(classifier, devtest_set)

0.759

In [27]:
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features1(name))
    if guess != tag:
        errors.append( (tag, guess, name) ) 

In [28]:
for (tag, guess, name) in sorted(errors): # doctest: +ELLIPSIS +NORMALIZE_WHITESPACE 
    print ('correct=%-8s guess=%-8s name=%-30s' % (tag, guess, name))

correct=female   guess=male     name=Abbey                         
correct=female   guess=male     name=Addis                         
correct=female   guess=male     name=Aeriel                        
correct=female   guess=male     name=Agnes                         
correct=female   guess=male     name=Alanah                        
correct=female   guess=male     name=Alyson                        
correct=female   guess=male     name=Amabel                        
correct=female   guess=male     name=Angel                         
correct=female   guess=male     name=Ariel                         
correct=female   guess=male     name=Arleen                        
correct=female   guess=male     name=Audry                         
correct=female   guess=male     name=Aurel                         
correct=female   guess=male     name=Beret                         
correct=female   guess=male     name=Brigit                        
correct=female   guess=male     name=Briney     